In [1]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import random
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def CNN(d,f,e):
    
    my_seed = 12345
    np.random.seed(my_seed)
    random.seed(my_seed)
    tf.random.set_seed(my_seed)

    y = to_categorical(f)
    
    model = Sequential()
    model.add(Dense(units=50, activation='relu', input_dim=d.shape[1]))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=y.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(d, y, epochs=e, shuffle=True, verbose=0)
    return model

In [3]:
# read data
df = pd.read_csv('../Data/combined_2018.csv')

In [4]:
# create moving averages
for i in range(31):
    colname = 'ma' + str(i+1)
    df[colname] = df.iloc[:,(i+8):(i+18)].sum(axis=1)

df.drop(df.columns[91:101], axis=1, inplace=True)

In [5]:
# Select trainging hour and test hour
data = df[df['hour'].isin([6,7,8])]
del df

In [6]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["ma31"].apply(setlabels)
data.shape

(45177, 93)

In [7]:
df_predict = pd.DataFrame()
profit_tot =[]

for j in range(42):
    
    start = 1000 * j

    # Get fitting data
    X_data = data.iloc[start:start + 3000].copy()
    X = X_data.iloc[:,67:91].to_numpy()
    y = data['labels'].iloc[start:start+3000]
    le = LabelEncoder()
    le.fit(y)
    y_encoded = le.transform(y)

    cnn = CNN(X, y_encoded, 1)

    # Get predicting data
    X_pred = data.iloc[start + 3100:start + 4100].copy()
    X_predict = X_pred.iloc[:,67:91].to_numpy()
    X_pred['class'] = le.inverse_transform(cnn.predict_classes(X_predict))
    X_pred = X_pred.drop(X_pred[(X_pred['class'] == 0)].index)
    df_predict = df_predict.append(X_pred)
    
    totalrows = X_pred.shape[0]
    totalcols = X_pred.shape[1]
    profit = []
    for i in range(totalrows):
        temp = X_pred.iloc[i,totalcols-1] * X_pred.iloc[i]['ma31']
        profit.append(temp)
    #print("Est Profit Cohort " + str(j) + ": " + str(sum(profit)))
    profit_tot.append(sum(profit))

print("Total Profit " + " : " + str(sum(profit_tot)))

Total Profit  : -133.5


In [8]:
df_predict.to_csv('2018_predict.csv')